In [4]:
# Install missing package in notebook environment
%pip install rapidfuzz

import pandas as pd
from rapidfuzz import process, fuzz
from tqdm import tqdm

# Load data
recipes = pd.read_csv("recipe2.csv")
walmart = pd.read_csv("walmart_with_coordinates_imputed.csv")

# Clean text
def clean_text(s):
    s = str(s).lower().strip()
    s = ''.join(ch for ch in s if ch.isalnum() or ch.isspace())
    return s

recipes["ingredient_clean"] = recipes["ingredient"].map(clean_text)
walmart["product_clean"] = walmart["PRODUCT_NAME"].map(clean_text)

# Prepare sets
choices = walmart["product_clean"].tolist()

# Cache & batching
unique_ingredients = recipes["ingredient_clean"].unique()
print(f"🧂 Unique ingredients to match: {len(unique_ingredients)}")

cache = {}
batch_size = 200

for start in tqdm(range(0, len(unique_ingredients), batch_size)):
    batch = unique_ingredients[start:start+batch_size]
    # vectorized distance computation
    results = process.cdist(
        batch, choices,
        scorer=fuzz.token_set_ratio,
        workers=-1,  # use all CPU cores
        score_cutoff=80
    )
    # get best match per ingredient
    for i, ing in enumerate(batch):
        if results[i].size > 0:
            best_idx = results[i].argmax()
            best_match = choices[best_idx]
            best_score = results[i][best_idx]
            cache[ing] = (best_match, best_score)
        else:
            cache[ing] = (None, None)

# Map cached matches to recipes
recipes["matched_product"] = recipes["ingredient_clean"].map(lambda x: cache[x][0])
recipes["match_score"] = recipes["ingredient_clean"].map(lambda x: cache[x][1])

# Merge
merged = recipes.merge(
    walmart,
    left_on="matched_product",
    right_on="product_clean",
    how="left",
    suffixes=("_recipe", "_walmart")
)

# Select relevant columns
final = merged[[
    "title", "ingredient", "amount", "unit",
    "matched_product", "match_score",
    "PRODUCT_NAME", "BRAND", "PRICE_RETAIL", "PRICE_CURRENT", "PRODUCT_SIZE",
    "latitude", "longitude", "SHIPPING_LOCATION", "city", "state_id" 
]]

# Export
final.to_csv("merged_recipe_prices_fast.csv", index=False)
print("Saved merged_recipe_prices_fast.csv")


Note: you may need to restart the kernel to use updated packages.


/var/folders/7p/xb2glmy50v59_g4kfskbtjz80000gn/T/ipykernel_7834/377507798.py:10: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  walmart = pd.read_csv("walmart_with_coordinates_imputed.csv")


🧂 Unique ingredients to match: 6251


100%|██████████| 32/32 [10:24<00:00, 19.50s/it]


Saved merged_recipe_prices_fast.csv


In [ ]:
import pandas as pd

# 1. 读取数据
merged = pd.read_csv("merged_recipe_prices_fast.csv")
print("Before deduplication:", len(merged))

# 2. 关键修改：去重逻辑
# 修改前：merged.drop_duplicates(subset=["title", "ingredient", "amount", "unit", "matched_product"])
# 修改后：把 'SHIPPING_LOCATION' 加入 subset，确保不同地点的同一种商品都被保留
merged_nodupl = merged.drop_duplicates(subset=[
    "title", "ingredient", "amount", "unit", "matched_product", "SHIPPING_LOCATION"
])

print("After deduplication (preserving locations):", len(merged_nodupl))

# 3. 选择列 (保留地理信息)
final_nodupl = merged_nodupl[[
    "title", "ingredient", "amount", "unit",
    "matched_product", "match_score",
    "PRODUCT_NAME", "BRAND", "PRICE_RETAIL", "PRICE_CURRENT", "PRODUCT_SIZE",
    "latitude", "longitude", "SHIPPING_LOCATION", "city", "state_id" 
]]

# 4. 导出最终文件
output_filename = "merged_recipe_price_nodupl.csv"
final_nodupl.to_csv(output_filename, index=False)
print(f" Saved {output_filename} (Ready for Location-Based Optimization)")

Before deduplication: 5621815
After deduplication (preserving locations): 2631005
✅ Saved merged_recipe_price_nodupl.csv (Ready for Location-Based Optimization)
